In [2]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import numpy as np
from scipy.linalg import sqrtm
from torchvision.models import inception_v3
import torchvision.transforms.functional as TF
from PIL import Image
import pandas as pd
import os
import multiprocessing

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import numpy as np
from scipy.linalg import sqrtm
from torchvision.models import inception_v3
from PIL import Image
import pandas as pd
import os
import torch.nn.functional as F
import multiprocessing
import joblib
from joblib import load
import torchvision.models as models
import csv

device = torch.device('mps')

In [18]:
# define the generator architecture (CHANGE THIS EVERY TIME)
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(channels)
        )

    def forward(self, x):
        return x + self.conv_block(x)
    
# ARCHITECTURE FOR MONET + BIG CATS
    
class Generator(nn.Module):
    def __init__(self, input_nc, output_nc):
        super(Generator, self).__init__()

        # Define the generator model using nn.Sequential
        self.model = nn.Sequential(
            # Initial convolution block
            nn.Conv2d(input_nc, 64, kernel_size=7, stride=2, padding=3, padding_mode='reflect'),
            nn.InstanceNorm2d(64),
            nn.ReLU(),
            
            # Downsampling
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(),
            
            # Residual blocks
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            
            # Upsampling
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, output_nc, kernel_size=7, stride=2, padding=3, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)
    
# class Generator(nn.Module):
#     def __init__(self, input_nc, output_nc):
#         super(Generator, self).__init__()

#         self.model = nn.Sequential(
#             # Encoder layers
#             nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1),
#             nn.ReLU(),
#             nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
#             nn.InstanceNorm2d(128),
#             nn.ReLU(),
#             # Residual blocks
#             ResidualBlock(128),
#             ResidualBlock(128),
#             ResidualBlock(128),
#             ResidualBlock(128),
#             ResidualBlock(128),
#             ResidualBlock(128),
#             # Decoder layers
#             nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
#             nn.InstanceNorm2d(64),
#             nn.ReLU(),
#             nn.ConvTranspose2d(64, output_nc, kernel_size=4, stride=2, padding=1),
#             nn.Tanh()
#         )

#     def forward(self, x):
#         return self.model(x)


In [19]:
def calculate_image_pair_metrics(real_images, fake_images):
    ssim_scores = []
    psnr_scores = []

    # Move images to CPU and convert to numpy for SSIM and PSNR calculations
    real_images = real_images.to('cpu').numpy()
    fake_images = fake_images.to('cpu').numpy()

    for real_img, fake_img in zip(real_images, fake_images):
        # Adjust format for SSIM and PSNR
        real_img_np = np.moveaxis(real_img, 0, -1)  # Convert from CxHxW to HxWxC
        fake_img_np = np.moveaxis(fake_img, 0, -1)

        # Specify win_size if necessary, here it's set as an example and may need to be adjusted
        win_size = min(7, real_img_np.shape[0], real_img_np.shape[1])  # Ensure win_size is appropriate

        # Calculate and collect metrics
        ssim_score = ssim(real_img_np, fake_img_np, data_range=real_img_np.max() - real_img_np.min(), multichannel=True, win_size=win_size, channel_axis=2)
        psnr_score = psnr(real_img_np, fake_img_np, data_range=real_img_np.max() - real_img_np.min())

        ssim_scores.append(ssim_score)
        psnr_scores.append(psnr_score)

    return ssim_scores, psnr_scores

In [15]:
# setting up the correct paths and dataloaders (THIS NEEDS TO BE CHANGED EVERY TIME)
epoch = 100
model_name = 'photo2monet'
checkpoint_path = f'/Users/kieran/Documents/mlpProject/runs/{model_name}/epoch_{epoch}.pth'
checkpoint = torch.load(checkpoint_path, map_location=device)

input_nc = 3  # Number of input channels
output_nc = 3  # Number of output channels
# Initialize networks
G_AB = Generator(input_nc, output_nc).to(device)
G_BA = Generator(output_nc, input_nc).to(device)

# Load the state dictionaries
G_AB.load_state_dict(checkpoint['G_AB_state_dict'])
G_BA.load_state_dict(checkpoint['G_BA_state_dict'])

# Ensure the models are in evaluation mode
G_AB.eval()
G_BA.eval()

# Prepare your dataset
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

train_dataset_A = ImageFolder(root='/Users/kieran/Documents/mlpProject/zebraHorse/trainA', transform=transform)
train_dataset_B = ImageFolder(root='/Users/kieran/Documents/mlpProject/zebraHorse/trainB', transform=transform)
test_dataset_A = ImageFolder(root='/Users/kieran/Documents/mlpProject/bigCats/test/TIGER', transform=transform)
test_dataset_B = ImageFolder(root='/Users/kieran/Documents/mlpProject/bigCats/test/TIGER', transform=transform)

train_dataloader_A = DataLoader(train_dataset_A, batch_size=8, shuffle=True)
train_dataloader_B = DataLoader(train_dataset_B, batch_size=8, shuffle=True)
test_dataloader_A = DataLoader(train_dataset_A, batch_size=8, shuffle=True)
test_dataloader_B = DataLoader(train_dataset_B, batch_size=8, shuffle=True)

# Generate and save/display images
output_dir = f'/Users/kieran/Documents/mlpProject/{model_name}/imageMetrics/{epoch}'
os.makedirs(output_dir, exist_ok=True)

In [16]:
def calculate_metrics_for_pair(G_AB, G_BA, device, dataloader_A, dataloader_B, save_dir, label):
    os.makedirs(save_dir, exist_ok=True)

    # Initialize metrics dictionary for the pair of dataloaders
    metrics = {'ssim_AB': [], 'psnr_AB': [], 'ssim_BA': [], 'psnr_BA': []}

    for _, (real_A_batch, _) in enumerate(dataloader_A):
        real_A_batch = real_A_batch.to(device)
        with torch.no_grad():
            fake_B_batch = G_AB(real_A_batch)
        ssim_AB, psnr_AB = calculate_image_pair_metrics(real_A_batch, fake_B_batch)
        metrics['ssim_AB'].extend(ssim_AB)
        metrics['psnr_AB'].extend(psnr_AB)

    for _, (real_B_batch, _) in enumerate(dataloader_B):
        real_B_batch = real_B_batch.to(device)
        with torch.no_grad():
            fake_A_batch = G_BA(real_B_batch)
        ssim_BA, psnr_BA = calculate_image_pair_metrics(real_B_batch, fake_A_batch)
        metrics['ssim_BA'].extend(ssim_BA)
        metrics['psnr_BA'].extend(psnr_BA)
    
    # Print the average metrics for the given pair of dataloaders
    print(f"Results for {label}:")
    for metric_name, scores in metrics.items():
        if scores:
            print(f"  {metric_name}: {np.mean(scores):.4f}")
        else:
            print(f"  {metric_name}: No Data")


# Example usage:
label_train = 'Train'
label_test = 'Test'
save_dir_train = os.path.join(output_dir, label_train)
save_dir_test = os.path.join(output_dir, label_test)

# Calculate and print metrics for the training datasets
calculate_metrics_for_pair(G_AB, G_BA, device, train_dataloader_A, train_dataloader_B, save_dir_train, label_train)

# Calculate and print metrics for the testing datasets
calculate_metrics_for_pair(G_AB, G_BA, device, test_dataloader_A, test_dataloader_B, save_dir_test, label_test)



Results for Train:
  ssim_AB: -0.1271
  psnr_AB: 8.7768
  ssim_BA: -0.1744
  psnr_BA: 8.5131
Results for Test:
  ssim_AB: -0.1271
  psnr_AB: 8.7768
  ssim_BA: -0.1744
  psnr_BA: 8.5131


In [21]:
from torchvision.transforms import Compose, Resize, Normalize, ToTensor, ToPILImage
from torchvision import models
import numpy as np
from scipy.linalg import sqrtm
import certifi
os.environ['SSL_CERT_FILE'] = certifi.where()
from torchvision.models import inception_v3

def calculate_kid_for_dataloaders(G_AB, device, dataloader_A, dataloader_B, label, num_batches=10):
    # vgg16 = models.vgg16(pretrained=True)
    # model_features = vgg16.features[:31]
    # for param in model_features.parameters():
    #     param.requires_grad = False
    # model_features.eval()
    # print("Model features loaded")

    inception = inception_v3(pretrained=True, transform_input=False).to(device)
    
    # Define preprocessing transformation for Inception V3
    preprocess = Compose([
        ToPILImage(),  # Convert tensors to PIL Images
        Resize((299, 299)),  # Resize the image for Inception V3
        ToTensor(),  # Convert images to Tensor
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize for Inception V3
    ])

    kid_A = []  # To store KID scores for each batch
    kid_B = []

    with torch.no_grad():
        for i, ((real_A_batch, _), (real_B_batch, _)) in enumerate(zip(dataloader_A, dataloader_B)):
            if i >= num_batches:  # Break after processing num_batches
                break
            # Apply preprocessing to real images
            real_A_batch = torch.stack([preprocess(image) for image in real_A_batch]).to(device)
            real_B_batch = torch.stack([preprocess(image) for image in real_B_batch]).to(device)
            # Generate fake images
            fake_A_batch = G_BA(real_B_batch)
            fake_B_batch = G_AB(real_A_batch)
            # Apply preprocessing to fake images - ensure they are detached from the graph
            fake_A_batch = torch.stack([preprocess(image.detach()) for image in fake_A_batch]).to(device)
            fake_B_batch = torch.stack([preprocess(image.detach()) for image in fake_B_batch]).to(device)

            # Assuming aux_logits=True, extract the main output explicitly
            real_features_A = inception(real_A_batch)[0].detach().cpu().numpy()
            real_features_B = inception(real_B_batch)[0].detach().cpu().numpy()
            fake_features_A = inception(fake_A_batch)[0].detach().cpu().numpy()
            fake_features_B = inception(fake_B_batch)[0].detach().cpu().numpy()

            # Calculate FID for the batch
            kid_A1 = calculate_fid_from_features(real_features_A, fake_features_A)
            kid_A.append(kid_A1)

            kid_B1 = calculate_fid_from_features(real_features_B, fake_features_B)
            kid_B.append(kid_B1)

    print(f"Average FID for {label} A: {np.mean(kid_A)}")
    print(f"Average FID for {label} B: {np.mean(kid_B)}")

def calculate_fid_from_features(real_features, fake_features):
    mu1, sigma1 = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu2, sigma2 = fake_features.mean(axis=0), np.cov(fake_features, rowvar=False)
    ssdiff = np.sum((mu1 - mu2) ** 2.0)
    covmean = sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

calculate_kid_for_dataloaders(G_AB, device, train_dataloader_A, train_dataloader_B, label_train)
calculate_kid_for_dataloaders(G_AB, device, test_dataloader_A, test_dataloader_B, label_test)

/Users/kieran/Documents/mlpProject/pytorch_env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kieran/Documents/mlpProject/pytorch_env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Average FID for Train A: 7.846375299196117e+55
Average FID for Train B: -5.88468378721426e+55
Average FID for Test A: -1.5652521419868583e+56
Average FID for Test B: -8.979469499653194e+51
